# MIE1624 Assignment 2

In [48]:
# importing:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [49]:
datasets = "datasets/"
df = pd.read_csv(datasets + "Kaggle_Salary.csv")
df.head()


,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,...,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT,Q10_Encoded,Q10_buckets
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,NaN,NaN
1,510,22-24,Male,-1,France,Master’s degree,Software Engineer,-1,"1000-9,999 employees",0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,3.0,"30,000-39,999"
2,423,40-44,Male,-1,India,Professional degree,Software Engineer,-1,"> 10,000 employees",20+,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0.0,"0-9,999"
3,391,40-44,Male,-1,Australia,Master’s degree,Other,0,"> 10,000 employees",20+,...,NaN,NaN,NaN,Azure SQL Database,NaN,NaN,NaN,-1,14.0,">250,000"
4,392,22-24,Male,-1,India,Bachelor’s degree,Other,1,0-49 employees,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,0.0,"0-9,999"


### Data Visualization

In [50]:
print("Number of rows =", df.shape[0])
print("Number of columns =", df.shape[1])

Number of rows = 12498
Number of columns = 248


## Data Cleaning
### Converting Categorical Data Into Numerical Data
There are a few options for encoding categorical data into numerical data. Given a categorical column we can choose to either encode each element numerically, encode the column using one-hot encoding, or encode the column using a custom one-hot encoding. 

Suppose the dataset in question is a dataset that contains information regarding which car each respondent owns. The column in question is ```car_type```. This column can take on 3 different values: ```tesla```, ```toyota```, and ```bmw```. The following will demonstrate three different types of encoding methods.

#### Numerical Encoding 
To numerically encode the ```car_type``` category we would assign the possible values that ```car_type``` could be as numerical values. We could encode ```tesla``` as 0, ```toyota``` as 1, and ```bmw``` as 2. The problem with this encoding method is the weighting implied by the numerical values. Are BMW's twice as valuable, heavy, or fast as Toyotas? If the data does not relate to the encoding in any way it is better to use one-hot encoding instead.

#### One-Hot Encoding
To encode a column using one-hot encoding we assign each possible value in a particular column its own column. Then, the values in each of these new columns is either 0 or 1 depending on if that particular entry possesses that property. ```tesla```, ```toyota```, and ```bmw``` would all be separate columns with boolean values in each entry. The drawback of one-hot encoding is the number of columns explodes with the number of unique values in each categorical column. In this rather simple example we added an additional 2 columns. If we repeated this process for a large number of categorical columns with many unique values in each column we would end up with a very large dataset that would take a long time to operate on. A potential solution is delinated by the next method.

#### Customized One-Hot Encoding
Suppose that in our example there are a total of 1000 respondents. 975 respondents own Teslas, 15 own BMWs, and 10 own Toyotas. In this case, it might be wiser to refactor our original ```car_type``` column to a more simple binary column like ```owns_tesla```. This would prevent an explosion of columns and would retain the attractive categorical implications of one-hot encoding. One could also situationally create a column called ```other``` that would encompass all of the other entires for a given dataset to refrain from adding several columns containing little information. Critically thinking about which encoding method you should use is essential for generating accurate models. 

### Column Visualization
Each column must be investigated to determine which encoding method should be used for each column.

In [51]:
question_dict = dict()

for i in range(df.shape[1]):
    if len(df.columns[i]) <= 3 or ("Part_1" in df.columns[i] and "Part_10" not in df.columns[i] and "Part_11" not in df.columns[i] and "Part_12" not in df.columns[i] and "Part_13" not in df.columns[i]):
        question_dict[df.columns[i]] =  df.iloc[0, i]

# Determining Which Questions Were Asked:
print("\nQuestions Asked in Dataset:\n")
for (key, value) in question_dict.items():
    print(key)
    print(value, "\n")



Questions Asked in Dataset:

Q1
What is your age (# years)? 

Q2
What is your gender? - Selected Choice 

Q3
In which country do you currently reside? 

Q4
What is the highest level of formal education that you have attained or plan to attain within the next 2 years? 

Q5
Select the title most similar to your current role (or most recent title if retired): - Selected Choice 

Q6
What is the size of the company where you are employed? 

Q7
Approximately how many individuals are responsible for data science workloads at your place of business? 

Q8
Does your current employer incorporate machine learning methods into their business? 

Q9_Part_1
Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Analyze and understand data to influence product or business decisions 

Q10
What is your current yearly compensation (approximate $USD)? 

Q11
Approximately how much money have you spent on machine learning and/or cloud computing

In [52]:
# Determining Which Questions Were Multiple Choice Questions:
print("\nMultiple Choice Questions in Dataset")
multiple_choice_dict = dict()
for (key, value) in question_dict.items():
    if "_" in key:
        multiple_choice_dict[key] = value
        print(key)
        print(value, "\n")



Multiple Choice Questions in Dataset
Q9_Part_1
Select any activities that make up an important part of your role at work: (Select all that apply) - Selected Choice - Analyze and understand data to influence product or business decisions 

Q12_Part_1
Who/what are your favorite media sources that report on data science topics? (Select all that apply) - Selected Choice - Twitter (data science influencers) 

Q13_Part_1
On which platforms have you begun or completed data science courses? (Select all that apply) - Selected Choice - Udacity 

Q14_Part_1_TEXT
What is the primary tool that you use at work or school to analyze data? (Include text response) - Basic statistical software (Microsoft Excel, Google Sheets, etc.) - Text 

Q16_Part_1
Which of the following integrated development environments (IDE's) do you use on a regular basis?  (Select all that apply) - Selected Choice - Jupyter (JupyterLab, Jupyter Notebooks, etc)  

Q17_Part_1
Which of the following hosted notebook products do you

### Data Cleaning Strategy
Multiple choice columns currently are populated by either a non-unique string that answers the particular categorical question or a NaN value. Consider Q9_Part_1, for example: 

In [53]:
print("Total number of rows =", df["Q9_Part_1"].shape[0])
print("Number of rows that are NaN =", df["Q9_Part_1"].shape[0] - df["Q9_Part_1"].dropna().count())
print("Each column value can either take on '{}' or '{}'".format(df["Q9_Part_1"].unique()[1], df["Q9_Part_1"].unique()[2]))

Total number of rows = 12498
Number of rows that are NaN = 6518
Each column value can either take on 'nan' or 'Analyze and understand data to influence product or business decisions'


For each of these columns we will fill the NaN entries (which imply negative responses) with 0's and we will replace all of the string values (which imply affirmative responses) with 1's. This will result columns that are one-hot encoded. 

In [54]:
def replace_strings(value):
    if value is not np.nan:
        return 1
    return 0

for column in df.columns:
    if "Text" in column:
        pass
    elif "Part" in column:
        df[column][1:] = df[column][1:].map(replace_strings) 

In [55]:
column_question_dict = dict()
for column in df.columns:
    column_question_dict[column] = df.iloc[0][column]
df.head()

,Time from Start to Finish (seconds),Q1,Q2,Q2_OTHER_TEXT,Q3,Q4,Q5,Q5_OTHER_TEXT,Q6,Q7,...,Q34_Part_6,Q34_Part_7,Q34_Part_8,Q34_Part_9,Q34_Part_10,Q34_Part_11,Q34_Part_12,Q34_OTHER_TEXT,Q10_Encoded,Q10_buckets
0,Duration (in seconds),What is your age (# years)?,What is your gender? - Selected Choice,What is your gender? - Prefer to self-describe...,In which country do you currently reside?,What is the highest level of formal education ...,Select the title most similar to your current ...,Select the title most similar to your current ...,What is the size of the company where you are ...,Approximately how many individuals are respons...,...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,Which of the following relational database pro...,NaN,NaN
1,510,22-24,Male,-1,France,Master’s degree,Software Engineer,-1,"1000-9,999 employees",0,...,0,0,0,0,0,0,0,-1,3.0,"30,000-39,999"
2,423,40-44,Male,-1,India,Professional degree,Software Engineer,-1,"> 10,000 employees",20+,...,0,0,0,0,0,0,0,-1,0.0,"0-9,999"
3,391,40-44,Male,-1,Australia,Master’s degree,Other,0,"> 10,000 employees",20+,...,0,0,0,1,0,0,0,-1,14.0,">250,000"
4,392,22-24,Male,-1,India,Bachelor’s degree,Other,1,0-49 employees,0,...,0,0,0,0,0,0,0,-1,0.0,"0-9,999"


Now that all of the columns that were originally categorical are usable, we will now investigate the one-hot encoded columns to see if any columns have less than 50 respondents that selected that column. Columns that have less than 50 selections will be merged into an "other" column for that particular question.

In [56]:
question_list = []
for column in df.columns:
    if "TEXT" in column:
        pass
    elif "Part" in column:
        print("Count for Question {} = {}".format(column[:2] if column[2] == "_" else column[:3], df[1:][column].sum()))

Count for Question Q9 = 5979
Count for Question Q9 = 3507
Count for Question Q9 = 4890
Count for Question Q9 = 3285
Count for Question Q9 = 3634
Count for Question Q9 = 2303
Count for Question Q9 = 515
Count for Question Q9 = 239
Count for Question Q12 = 2785
Count for Question Q12 = 1386
Count for Question Q12 = 2404
Count for Question Q12 = 7757
Count for Question Q12 = 2817
Count for Question Q12 = 5116
Count for Question Q12 = 1644
Count for Question Q12 = 7333
Count for Question Q12 = 3494
Count for Question Q12 = 1881
Count for Question Q12 = 374
Count for Question Q12 = 883
Count for Question Q13 = 2278
Count for Question Q13 = 6487
Count for Question Q13 = 2439
Count for Question Q13 = 2978
Count for Question Q13 = 557
Count for Question Q13 = 3666
Count for Question Q13 = 1190
Count for Question Q13 = 3561
Count for Question Q13 = 1010
Count for Question Q13 = 3062
Count for Question Q13 = 994
Count for Question Q13 = 1277
Count for Question Q16 = 7815
Count for Question Q16 =

Since each categorical column has more than 50 selections no columns will be combined. Now that we have one-hot encoded the already multiple choice data we will deal with the rest of the columns present in the dataset. 

#### "TEXT" and "OTHER_TEXT" Columns
The "TEXT" and "OTHER_TEXT" columns are columns that encode each respondent's reply with a numerical encoding. Respondent's with a similar reply have the same encoding. We run into the same problem we typically run into when using numerical encoding: higher numerical encoding values aren't indicative of some implicit greater quantity. How will the model interpret a response encoding of 134 versus a response encoding of 4? 

Instead of one-hot encoding all of the unique responses we will instead chooes to ignore the "TEXT" and "OTHER_TEXT" columns. We will do this because the information presented by these columns is mostly captured by the "Other" columns preceeding these text columns.

In [57]:
for column in df.columns:
    if "TEXT" in column or "OTHER_TEXT" in column:
        print(column)
        df.drop([column], axis=1, inplace=True)

Q2_OTHER_TEXT
Q5_OTHER_TEXT
Q9_OTHER_TEXT
Q12_OTHER_TEXT
Q13_OTHER_TEXT
Q14_Part_1_TEXT
Q14_Part_2_TEXT
Q14_Part_3_TEXT
Q14_Part_4_TEXT
Q14_Part_5_TEXT
Q14_OTHER_TEXT
Q16_OTHER_TEXT
Q17_OTHER_TEXT
Q18_OTHER_TEXT
Q19_OTHER_TEXT
Q20_OTHER_TEXT
Q21_OTHER_TEXT
Q24_OTHER_TEXT
Q25_OTHER_TEXT
Q26_OTHER_TEXT
Q27_OTHER_TEXT
Q28_OTHER_TEXT
Q29_OTHER_TEXT
Q30_OTHER_TEXT
Q31_OTHER_TEXT
Q32_OTHER_TEXT
Q33_OTHER_TEXT
Q34_OTHER_TEXT


Now we must deal with the rest of the categorical columns before we begin investigating the impacts of each column. 

#### Age Range Encoding (Q1)
For this column each response is an age range. Therefore, a clear relationship exists between the various ranges. We could capture this relationship by numerically encoding each range. 

#### Gender Encoding (Q2)
There are four different values that an element in the gender column can take on: "Male", "Female", "Prefer to self-describe", and "Prefer not to say". The vast majorty of respondents are "Male" or "Female". Using one-hot encoding we will define three columns: "Male", "Female", and "Other". In this way we save a column without sacrificing too much information.

#### Country (Q3)
Since there are so many countries it doesn't make sense to one-hot encode all of the countries. Our dataset would explode in size and the information gained from this explosion would not be worth it. Instead of dropping the column we will atempt to find some sort of middle ground. The 5 most popular countries will be encoded and the rest of the countries will be combined into an "Other" column.

#### Highest Level of Formal Education (Q4)
We can organize the values in this column using numerical encoding because we can consider each education level as a education "tier". Higher numerical values correspond with higher education tiers. 

#### Current Role (Q5)
There are too many factors to consider when rank ordering occupations by any criteria. Therefore, we will one-hot encode this column. The increase in the number of columns won't be too much of a problem because the number of possible positions is relatively small. 

#### Company Size (Q6)
The possible responses delineated by this column are range values. We can loosely rank these ranges using numerical encoding. 

#### Number of Employees Responsible for Data Science Workload (Q7)
Since this column contains ranges of employees it makes the most sense to numerically encode these values. 

#### Company Machine Learning (Q8)
This column indicates whether a company uses machine learning in their business and if so, how recently have they incorporated it. There are few possible values including "No", "I do not know", "We are exploring...", "We have recently established...", and "We have well established...". Given that there are not many possible replies we will one-hot encode this column

#### Money Spent on Machine Learning (Q11)
This column includes information regarding money spent on machine learning products in the past 5 years. Given that ranges were provided these values will be numerically encoded. 

#### Analysis Tools (Q14)
The recorded responses provide insight into which analysis tools the respondents prefer. These were originally encoded using one-hot encoding and then further encoded using numerical encoding to further desribe the similarities between responses. They will be re-encoded using one-hot encoding because the similarities between responses is difficult to interpret without one-hot encoding every single response as its own column.

#### Coding Experience (Q15)
Numerical encoding is the obvious choice for this column because the coding experience of each respondent is measured using ranges. The relationship between these ranges can be linearly approximated by assigning integers to each range. 

#### Recommend an Introductory Coding Language (Q19)
Respondents were asked to recommend a programming language to an aspiring data scientist. The vast majority of replies were "Python", but many other languages were also recommended. This column will also be one-hot encoded. The NaN values in this column will be replaced with "Other" because they were implicitly defined as "Other" in subsequent column.

#### TPU Experience (Q22) & Machine Learning Experience (Q23)

These two columns will be numerically encoded because a relationship exists between the ranges defined by the surveryor. The assigned integer values will attempt to capture this relationship. 

*NOTE*: Interestingly, there are some NaN values for the columns we wish to numerically encode, namely, Q11, Q15, Q22, and Q23. We will investigate how many NaN's are present in each column and then we will make a decision on what to do about these NaN values. 

In [58]:
df = df[1:]

# one-hot columns:
one_hot_columns = ["Q2", "Q3", "Q5", "Q8", "Q14", "Q19"]

# numerical columns:
numerical_columns = ["Q1", "Q4", "Q6", "Q7", "Q11", "Q15", "Q22", "Q23"]

for column in one_hot_columns:
    df = pd.concat([df, pd.get_dummies(df[column], prefix=column)], axis=1)
    df.drop([column], axis=1, inplace=True)
df.head()

,Time from Start to Finish (seconds),Q1,Q4,Q6,Q7,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,...,Q19_C++,Q19_Java,Q19_Javascript,Q19_MATLAB,Q19_None,Q19_Other,Q19_Python,Q19_R,Q19_SQL,Q19_TypeScript
1,510,22-24,Master’s degree,"1000-9,999 employees",0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,423,40-44,Professional degree,"> 10,000 employees",20+,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,391,40-44,Master’s degree,"> 10,000 employees",20+,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,392,22-24,Bachelor’s degree,0-49 employees,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,470,50-54,Master’s degree,0-49 employees,3-4,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [59]:
for column in numerical_columns:
    print(column)
    print(df[column].unique())

Q1
['22-24' '40-44' '50-54' '55-59' '30-34' '18-21' '35-39' '25-29' '45-49'
 '60-69' '70+']
Q4
['Master’s degree' 'Professional degree' 'Bachelor’s degree'
 'Doctoral degree'
 'Some college/university study without earning a bachelor’s degree'
 'I prefer not to answer' 'No formal education past high school']
Q6
['1000-9,999 employees' '> 10,000 employees' '0-49 employees'
 '50-249 employees' '250-999 employees']
Q7
['0' '20+' '3-4' '1-2' '5-9' '10-14' '15-19']
Q11
['$0 (USD)' '> $100,000 ($USD)' '$10,000-$99,999' '$100-$999'
 '$1000-$9,999' '$1-$99' nan]
Q15
['1-2 years' 'I have never written code' '< 1 years' '20+ years'
 '3-5 years' '5-10 years' '10-20 years' nan]
Q22
['Never' nan 'Once' '6-24 times' '2-5 times' '> 25 times']
Q23
['1-2 years' nan '2-3 years' '< 1 years' '10-15 years' '3-4 years'
 '4-5 years' '5-10 years' '20+ years']


In [60]:

q1_map = {"Q1": {'18-21': 0, '22-24': 1, '25-29': 2, '30-34': 3, '35-39': 4, '40-44': 5, '45-49': 6, '50-54': 7, '55-59': 8, '60-69': 9, '70+': 10}}
df = df.replace(q1_map)

q4_map = {"Q4": {"I prefer not to answer": 0, "No formal education past high school": 0, "Some college/university study without earning a bachelor’s degree": 1, "Professional degree": 2, "Bachelor’s degree": 3, "Master’s degree": 4, "Doctoral degree": 5}}
df = df.replace(q4_map)

q6_map = {"Q6": {"0-49 employees": 0, '50-249 employees': 1, '250-999 employees': 2, '1000-9,999 employees': 3, '> 10,000 employees': 4}}
df = df.replace(q6_map)

q7_map = {"Q7": {'0': 0, '1-2': 1, '3-4': 2, '5-9': 3, '10-14': 4, '15-19': 5, '20+': 6}}
df = df.replace(q7_map)

# For the mode of Q11
print(df["Q11"].describe(), "\n")
# nan's exist
q11_map = {"Q11": {'$0 (USD)': 0, '$1-$99': 1, '$100-$999': 2, '$1000-$9,999': 3, '$10,000-$99,999': 4, '> $100,000 ($USD)': 5}}
df = df.replace(q11_map)

# For the mode of Q15
print(df["Q15"].describe())
# nan's exist
q15_map = {"Q15": {'I have never written code': 0, '< 1 years': 1, '1-2 years': 2, '3-5 years': 3, '5-10 years': 4, '10-20 years': 5, '20+ years': 6}}
df = df.replace(q15_map)

# nan's exist
q22_map = {"Q22": {'Never': 0, 'Once': 1, '2-5 times': 2, '6-24 times': 3, '> 25 times': 4}}
df = df.replace(q22_map)

# nan's exist
q23_map = {"Q23": {'< 1 years': 0, '1-2 years': 1, '2-3 years': 2, '3-4 years': 3, '4-5 years': 4, '5-10 years': 5, '10-15 years': 6, '20+ years': 8}}
df = df.replace(q23_map)
df.head()




count        12250
unique           6
top       $0 (USD)
freq          4038
Name: Q11, dtype: object 

count         11422
unique            7
top       3-5 years
freq           2818
Name: Q15, dtype: object


,Time from Start to Finish (seconds),Q1,Q4,Q6,Q7,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,...,Q19_C++,Q19_Java,Q19_Javascript,Q19_MATLAB,Q19_None,Q19_Other,Q19_Python,Q19_R,Q19_SQL,Q19_TypeScript
1,510,1,4,3,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,423,5,2,4,6,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,391,5,4,4,6,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,392,1,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,470,7,4,0,2,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


In [61]:
print(df["Q11"].dropna().count())
print(df["Q15"].dropna().count())
print(df["Q22"].dropna().count())
print(df["Q23"].dropna().count())

12250
11422
10550
10541


By dropping all of the NaN values we lose about 1.7k survey results. This is too many results to sacrifice. We could choose to set all of the NaN values to the mode of the categorical data, however, give the sheer magnitude of the NaN values we would just be enforcing the mode and creating potentially statistically significant results based off of nothing. The columns that contain the most amount of NaN values are Q22 and Q23. We will drop these columns and replace the NaN values present in the other numerically encoded data with the modes of each column. 

In [62]:
df.drop(["Q22", "Q23"], axis=1, inplace=True)

# Replacing NaN values present in Q11 and Q15 with their respective modes:
df["Q11"] = df["Q11"].replace(np.nan, 0)
df["Q15"] = df["Q15"].replace(np.nan, 3)

# df.to_csv(datasets + "parsed_dataset.csv")

df.head()

,Time from Start to Finish (seconds),Q1,Q4,Q6,Q7,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,...,Q19_C++,Q19_Java,Q19_Javascript,Q19_MATLAB,Q19_None,Q19_Other,Q19_Python,Q19_R,Q19_SQL,Q19_TypeScript
1,510,1,4,3,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,423,5,2,4,6,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,391,5,4,4,6,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,392,1,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5,470,7,4,0,2,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0


There are now no NaN values left in the dataset and our entire dataset is numerical with the exception of our target variable, Q10. Note that Q10 also has encoded forms still present in the dataset. We can now begin to hypothesize relationships between the variables in the dataset. 

## Exploratory Data Analysis and Feature Selection
For this section we will select features to be used for our regression analysis. To do this, we will first investigate the data to uncover linear relationships between the dependent variables (all columns other than Q10) and the independent variable, Q10. The metric that we will use to determine the strength of the linear relationship between the dependent variables and the independent variable is the Pearson product-moment correlation value, or, r-value. The $r$ value is the ratio of the covariance of the variable pair to the products of the standard deviations of the variable pair. 

$COV(x, y) = \frac{\sum_{i=1}^{N}(x_i - \overline{x})(y_i - \overline{y})}{N}$

$s_x = \sqrt{\frac{\sum_{i=1}^{N}(x_i - \overline{x})^2}{N}}, s_y = \sqrt{\frac{\sum_{i=1}^{N}(y_i - \overline{y})^2}{N}}$

$r_{x, y} = \frac{COV(x, y)}{s_x \cdot s_y}$

$r_{x, y} = \frac{\sum_{i=1}^{N}(x_i - \overline{x})(y_i - \overline{y})}{\sqrt{\sum_{i=1}^{N}(x_i - \overline{x})^2}\sqrt{\sum_{i=1}^{N}(y_i - \overline{y})^2}}$

This metric measures the strength of the linear correlation between two IID variables. $r$ values that are closer to 1 indicate a positive linear relationship, $r$ values close to 0 indicate that a weak or no linear relationship exists, and $r$ values close to -1 indicate that a negative linear relationship exists. Creating a matrix of $r$ values pertaining to the various independent variables is a good way to quickly discover which variables are linearly correlated with the output variable. 

We will create an $N x 1$ matrix of r-values where each entry corresponds with the r-value of the dependent variable in question and the independent target variable.